#Import Dataset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import FactorAnalysis
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, StratifiedKFold
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os

root_dir = "/content/gdrive/MyDrive/Colab Notebooks/2-1 패턴인식"

# Checking if our specified directory exists
os.path.exists(root_dir)

True

In [ ]:
#train
data_path = os.path.join(root_dir, "train.csv")
df_train = pd.read_csv(data_path)

#test
data_path = os.path.join(root_dir, "test.csv")
df_test = pd.read_csv(data_path)

In [ ]:
df_train

,id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,12224,46,admin.,married,high.school,unknown,no,no,telephone,jul,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.966,5228.1,no
1,36387,32,admin.,married,university.degree,no,yes,yes,cellular,jun,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2,yes
2,22703,36,technician,single,university.degree,no,no,no,cellular,aug,...,5,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,no
3,32742,30,services,single,high.school,no,yes,yes,cellular,may,...,1,999,2,failure,-1.8,92.893,-46.2,1.299,5099.1,no
4,40547,76,retired,divorced,basic.4y,no,no,no,cellular,aug,...,1,999,1,failure,-1.7,94.027,-38.3,0.886,4991.6,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,24109,37,admin.,married,university.degree,no,yes,no,telephone,nov,...,1,6,1,success,-0.1,93.200,-42.0,4.286,5195.8,no
32946,14149,56,unknown,single,basic.9y,no,no,yes,cellular,jul,...,5,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
32947,12379,55,admin.,married,high.school,no,yes,no,cellular,jul,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32948,10138,34,services,married,high.school,no,no,no,telephone,jun,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.958,5228.1,no


#y 빼고 중복값 확인 후 yes 남기기

In [ ]:
df_train_ini = pd.read_csv(data_path)

In [ ]:
df_train_ini.duplicated().sum()

0

In [ ]:
df_train.drop_duplicates(inplace=True)

In [ ]:
columns_without_ynid = [col for col in df_train.columns if ((col != 'y') and (col!= 'id'))]
columns_without_ynid

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed']

In [ ]:
df_train[columns_without_ynid].duplicated(keep=False).sum()

2504

In [ ]:
df_test[columns_without_ynid].duplicated(keep=False).sum()

181

In [ ]:
train_duplicated = df_train.loc[df_train[columns_without_ynid].duplicated(keep=False), :]

In [ ]:
train_duplicated['y'].value_counts()

y
no     2290
yes     214
Name: count, dtype: int64

In [ ]:
idx = train_duplicated[train_duplicated['y']=='no']['id']

idx = idx.to_list()
type(idx) #list
type(idx[0]) #int

int

In [ ]:
df_train = df_train[~df_train['id'].isin(idx)]
df_train

,id,age,job,marital,education,default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,12224,46,admin.,married,high.school,unknown,no,no,telephone,jul,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.966,5228.1,no
1,36387,32,admin.,married,university.degree,no,yes,yes,cellular,jun,...,1,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2,yes
2,22703,36,technician,single,university.degree,no,no,no,cellular,aug,...,5,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,no
3,32742,30,services,single,high.school,no,yes,yes,cellular,may,...,1,999,2,failure,-1.8,92.893,-46.2,1.299,5099.1,no
4,40547,76,retired,divorced,basic.4y,no,no,no,cellular,aug,...,1,999,1,failure,-1.7,94.027,-38.3,0.886,4991.6,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,24109,37,admin.,married,university.degree,no,yes,no,telephone,nov,...,1,6,1,success,-0.1,93.200,-42.0,4.286,5195.8,no
32946,14149,56,unknown,single,basic.9y,no,no,yes,cellular,jul,...,5,999,0,nonexistent,1.4,93.918,-42.7,4.962,5228.1,no
32947,12379,55,admin.,married,high.school,no,yes,no,cellular,jul,...,3,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32948,10138,34,services,married,high.school,no,no,no,telephone,jun,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.958,5228.1,no


In [ ]:
# Encode the Categorical Variables.

df_train["month"] = df_train["month"].replace({"jul": 7, "jun": 6, "aug": 8, "may": 5, "oct" : 10, 'sep': 9, "nov": 11, "apr": 4, "mar":3, "dec": 12})

df_train["year"] = df_train["cons.price.idx"]
df_train["year"] = df_train["cons.price.idx"].replace({93.918 : 2008, 92.963 : 2009, 93.444 : 2008, 92.893 : 2009,
                                                       94.027 : 2010, 92.431 : 2009, 94.199 : 2010, 94.465 : 2008,
                                                       92.649 : 2009, 93.994 : 2008, 93.075 : 2009, 93.2 : 2008,
                                                       94.601 : 2010, 92.469 : 2009, 92.201 : 2009, 93.369 : 2010,
                                                       93.876 : 2010, 92.713 : 2009, 93.798 : 2008, 92.843 : 2009,
                                                       92.379 : 2009, 94.055 : 2010, 94.215 : 2010, 93.749 : 2010,
                                                       94.767 : 2010, 92.756 : 2008})

<ipython-input-18-9e1d80390d34>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["month"] = df_train["month"].replace({"jul": 7, "jun": 6, "aug": 8, "may": 5, "oct" : 10, 'sep': 9, "nov": 11, "apr": 4, "mar":3, "dec": 12})
<ipython-input-18-9e1d80390d34>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["year"] = df_train["cons.price.idx"]
<ipython-input-18-9e1d80390d34>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [ ]:
df_train['job'].replace('unknown', df_train['job'].mode().iloc[0], inplace=True)

<ipython-input-20-ceff3c0a97e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['job'].replace('unknown', df_train['job'].mode().iloc[0], inplace=True)


In [ ]:
df_train['loan'].replace('unknown', df_train['loan'].mode().iloc[0], inplace=True)

<ipython-input-21-a17d5892cb4d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['loan'].replace('unknown', df_train['loan'].mode().iloc[0], inplace=True)


In [ ]:
df_train['marital'].replace('unknown', df_train['marital'].mode().iloc[0], inplace=True)

<ipython-input-22-db0740cd669e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['marital'].replace('unknown', df_train['marital'].mode().iloc[0], inplace=True)


In [ ]:
conti_var_new =df_train.describe().columns

In [ ]:
df_train = df_train.drop(labels=["euribor3m", "nr.employed"], axis=1)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30660 entries, 0 to 32949
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              30660 non-null  int64  
 1   age             30660 non-null  int64  
 2   job             30660 non-null  object 
 3   marital         30660 non-null  object 
 4   education       30660 non-null  object 
 5   default         30660 non-null  object 
 6   housing         30660 non-null  object 
 7   loan            30660 non-null  object 
 8   contact         30660 non-null  object 
 9   month           30660 non-null  int64  
 10  day_of_week     30660 non-null  object 
 11  campaign        30660 non-null  int64  
 12  pdays           30660 non-null  int64  
 13  previous        30660 non-null  int64  
 14  poutcome        30660 non-null  object 
 15  emp.var.rate    30660 non-null  float64
 16  cons.price.idx  30660 non-null  float64
 17  cons.conf.idx   30660 non-null  floa

#make Dummy variables

In [ ]:
columns = df_train.select_dtypes(include=[object]).columns
df = pd.concat([df_train, pd.get_dummies(df_train[columns])], axis=1)
df = df.drop(['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week','poutcome', 'y'], axis =1)
print("\n","Describe Dataset_v2","\n")
print(df.info(),"\n \n \n")


 Describe Dataset_v2 

<class 'pandas.core.frame.DataFrame'>
Index: 30660 entries, 0 to 32949
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             30660 non-null  int64  
 1   age                            30660 non-null  int64  
 2   campaign                       30660 non-null  int64  
 3   pdays                          30660 non-null  int64  
 4   previous                       30660 non-null  int64  
 5   emp.var.rate                   30660 non-null  float64
 6   cons.price.idx                 30660 non-null  float64
 7   cons.conf.idx                  30660 non-null  float64
 8   year                           30660 non-null  float64
 9   job_admin.                     30660 non-null  bool   
 10  job_blue-collar                30660 non-null  bool   
 11  job_entrepreneur               30660 non-null  bool   
 12  job_housemaid              

In [ ]:
df = df.drop('default_yes', axis = 1)

In [ ]:
df.columns

Index(['id', 'age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'year', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_basic.4y',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'default_no',
       'default_unknown', 'housing_no', 'housing_unknown', 'housing_yes',
       'loan_no', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success', 'y_no', 'y_yes'],
      dtype='object')

#Scaling, Train-Test Split, Downsampling

In [ ]:
min_max_scaler=preprocessing.MinMaxScaler()
data_scaled=pd.DataFrame(min_max_scaler.fit_transform(df),columns=df.columns)

In [ ]:
y = data_scaled.y_yes
data_scaled = data_scaled.drop(['y_yes','y_no'], axis = 1)

In [ ]:
df = df.drop(['y_yes','y_no'], axis = 1)

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import NeighbourhoodCleaningRule

In [ ]:
from imblearn.under_sampling import NeighbourhoodCleaningRule

# NeighbourhoodCleaningRule을 사용하여 데이터 샘플링
X_samp, y_samp = NeighbourhoodCleaningRule(kind_sel="mode", n_neighbors=7).fit_resample(data_scaled, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_samp, y_samp, shuffle=True, test_size=0.2, random_state=42)

#Training & Validation

In [ ]:
!pip install catboost
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
# CatBoost 모델 정의
model = CatBoostClassifier()

#model training
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

Learning rate set to 0.033205
0:	learn: 0.6641491	total: 7.38ms	remaining: 7.37s
1:	learn: 0.6377442	total: 13.7ms	remaining: 6.82s
2:	learn: 0.6136060	total: 19.6ms	remaining: 6.51s
3:	learn: 0.5911146	total: 27.4ms	remaining: 6.83s
4:	learn: 0.5708027	total: 33.3ms	remaining: 6.63s
5:	learn: 0.5514431	total: 39.3ms	remaining: 6.51s
6:	learn: 0.5341867	total: 45.2ms	remaining: 6.41s
7:	learn: 0.5184364	total: 51.1ms	remaining: 6.33s
8:	learn: 0.5036770	total: 56.9ms	remaining: 6.27s
9:	learn: 0.4904850	total: 61.8ms	remaining: 6.12s
10:	learn: 0.4778117	total: 67.9ms	remaining: 6.1s
11:	learn: 0.4668547	total: 73.6ms	remaining: 6.06s
12:	learn: 0.4566819	total: 79.4ms	remaining: 6.03s
13:	learn: 0.4467269	total: 85.4ms	remaining: 6.02s
14:	learn: 0.4374252	total: 91.7ms	remaining: 6.02s
15:	learn: 0.4296899	total: 97.8ms	remaining: 6.02s
16:	learn: 0.4221040	total: 104ms	remaining: 6.03s
17:	learn: 0.4144123	total: 110ms	remaining: 6.01s
18:	learn: 0.4080156	total: 116ms	remaining: 6s

#Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

y_prob = model.predict_proba(X_test)
print("Estimated probs:", y_prob[:10])

y_cls = model.predict(X_test)
print("Estimated classes:", y_cls[:10])
print()

Estimated probs: [[0.95261015 0.04738985]
 [0.9293153  0.0706847 ]
 [0.93640435 0.06359565]
 [0.86680991 0.13319009]
 [0.91616138 0.08383862]
 [0.93690833 0.06309167]
 [0.90631698 0.09368302]
 [0.95923166 0.04076834]
 [0.87587685 0.12412315]
 [0.99130425 0.00869575]]
Estimated classes: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



In [ ]:
print("Accuracy:", accuracy_score(y_test, y_cls))
print("F1:", f1_score(y_test, y_cls))
print("ROC AUC:", roc_auc_score(y_test, y_prob[:, 1]))

Accuracy: 0.8975483870967742
F1: 0.6732510288065843
ROC AUC: 0.8735368191523997


In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_cls)
conf_matrix_df = pd.DataFrame(
    conf_matrix,
    columns=["Predicted yes", "Predicted no"],
    index=["Actual yes", "Actual no"]
)
print(conf_matrix_df)

            Predicted yes  Predicted no
Actual yes           3069            93
Actual no             304           409


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_cls))

              precision    recall  f1-score   support

         0.0       0.91      0.97      0.94      3162
         1.0       0.81      0.57      0.67       713

    accuracy                           0.90      3875
   macro avg       0.86      0.77      0.81      3875
weighted avg       0.89      0.90      0.89      3875



#Test with answer set

In [ ]:
import os

root_dir = "/content/gdrive/MyDrive/Colab Notebooks/"

# Checking if our specified directory exists
os.path.exists(root_dir)

True

In [ ]:
# Paths to the downloaded files
data_path = os.path.join(root_dir, "merged_with_answer (1).csv")

# Load data
df_test = pd.read_csv(data_path)

In [ ]:
df_test.head()

,Unnamed: 0,age,job,marital,education,default,housing,loan,contact,month,...,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,id,key,y
0,0,54,admin.,married,university.degree,no,no,no,cellular,nov,...,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,24188,54admin.marrieduniversity.degreenononocellular...,no
1,1,36,entrepreneur,married,university.degree,no,yes,yes,telephone,jun,...,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1,8688,36entrepreneurmarrieduniversity.degreenoyesyes...,no
2,2,27,unemployed,married,high.school,no,yes,no,cellular,may,...,0,nonexistent,-1.8,92.893,-46.2,1.244,5099.1,35789,27unemployedmarriedhigh.schoolnoyesnocellularm...,no
3,3,39,admin.,married,university.degree,no,yes,no,cellular,aug,...,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,20034,39admin.marrieduniversity.degreenoyesnocellula...,no
4,4,43,services,married,high.school,unknown,yes,no,telephone,jun,...,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,11392,43servicesmarriedhigh.schoolunknownyesnoteleph...,no


In [ ]:
df_test = df_test.drop(['Unnamed: 0', 'key'], axis =1)

In [ ]:
df_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,id,y
0,54,admin.,married,university.degree,no,no,no,cellular,nov,mon,...,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,24188,no
1,36,entrepreneur,married,university.degree,no,yes,yes,telephone,jun,wed,...,999,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1,8688,no
2,27,unemployed,married,high.school,no,yes,no,cellular,may,mon,...,999,0,nonexistent,-1.8,92.893,-46.2,1.244,5099.1,35789,no
3,39,admin.,married,university.degree,no,yes,no,cellular,aug,mon,...,999,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,20034,no
4,43,services,married,high.school,unknown,yes,no,telephone,jun,fri,...,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,11392,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8343,30,services,married,high.school,no,yes,no,telephone,nov,thu,...,999,0,nonexistent,-0.1,93.200,-42.0,4.076,5195.8,26975,no
8344,47,admin.,married,high.school,no,yes,no,telephone,may,wed,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,6863,no
8345,37,entrepreneur,married,basic.9y,no,no,no,telephone,jun,fri,...,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,9449,no
8346,47,services,married,high.school,unknown,yes,no,telephone,may,wed,...,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,916,no


In [ ]:
columns = df_test.select_dtypes(include=[object]).columns
df1 = pd.concat([df_test, pd.get_dummies(df_test[columns])], axis=1)
df1 = df1.drop(['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week','poutcome', 'y'], axis =1)
print("\n","Describe Dataset_v2","\n")
print(df1.info(),"\n \n \n")


 Describe Dataset_v2 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8348 entries, 0 to 8347
Data columns (total 64 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            8348 non-null   int64  
 1   campaign                       8348 non-null   int64  
 2   pdays                          8348 non-null   int64  
 3   previous                       8348 non-null   int64  
 4   emp.var.rate                   8348 non-null   float64
 5   cons.price.idx                 8348 non-null   float64
 6   cons.conf.idx                  8348 non-null   float64
 7   euribor3m                      8348 non-null   float64
 8   nr.employed                    8348 non-null   float64
 9   id                             8348 non-null   int64  
 10  job_admin.                     8348 non-null   bool   
 11  job_blue-collar                8348 non-null   bool   
 12  job_entrepreneur        

In [ ]:
df1.head()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,id,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,54,1,999,0,-0.1,93.200,-42.0,4.191,5195.8,24188,...,False,True,False,False,False,False,True,False,True,False
1,36,1,999,0,1.4,94.465,-41.8,4.864,5228.1,8688,...,False,False,False,False,True,False,True,False,True,False
2,27,4,999,0,-1.8,92.893,-46.2,1.244,5099.1,35789,...,False,True,False,False,False,False,True,False,True,False
3,39,4,999,0,1.4,93.444,-36.1,4.965,5228.1,20034,...,False,True,False,False,False,False,True,False,True,False
4,43,2,999,0,1.4,94.465,-41.8,4.959,5228.1,11392,...,True,False,False,False,False,False,True,False,True,False


In [ ]:
min_max_scaler=preprocessing.MinMaxScaler()
data_scaled1=pd.DataFrame(min_max_scaler.fit_transform(df1),columns=df1.columns)

In [ ]:
yt = data_scaled1.y_yes
xt = data_scaled1.drop(['y_yes','y_no'], axis = 1)

In [ ]:
xt.columns

Index(['age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'id',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'housing_no',
       'housing_unknown', 'housing_yes', 'loan_no', 'loan_unknown', 'loan_yes',
       'contact_cellular', 'contact_telephone', 'month_apr', 'month_aug',
       'month_dec', 'month_jul', 'month_jun', 'month_mar', 'month_may',
       'month_nov', 'month_oct', 'month_sep', 'day

In [ ]:
df.columns

Index(['id', 'age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'year', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_basic.4y',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'default_no',
       'default_unknown', 'housing_no', 'housing_unknown', 'housing_yes',
       'loan_no', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [ ]:
xt = xt[['id', 'age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success']]

In [ ]:
!pip install catboost
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
# CatBoost 모델 정의
model = CatBoostClassifier()

#model training
model.fit(X_train, y_train)

y_final = model.predict(xt)

# F1 점수 계산
f1 = f1_score(yt, y_final)
print("F1 Score:", f1)

Learning rate set to 0.033205
0:	learn: 0.6641491	total: 6.16ms	remaining: 6.15s
1:	learn: 0.6377442	total: 12.5ms	remaining: 6.25s
2:	learn: 0.6136060	total: 18.3ms	remaining: 6.09s
3:	learn: 0.5911146	total: 24.1ms	remaining: 6s
4:	learn: 0.5708027	total: 29.8ms	remaining: 5.93s
5:	learn: 0.5514431	total: 36.8ms	remaining: 6.09s
6:	learn: 0.5341867	total: 43.7ms	remaining: 6.2s
7:	learn: 0.5184364	total: 49.6ms	remaining: 6.15s
8:	learn: 0.5036770	total: 55.3ms	remaining: 6.08s
9:	learn: 0.4904850	total: 60.2ms	remaining: 5.96s
10:	learn: 0.4778117	total: 65.9ms	remaining: 5.92s
11:	learn: 0.4668547	total: 71.5ms	remaining: 5.88s
12:	learn: 0.4566819	total: 77.1ms	remaining: 5.85s
13:	learn: 0.4467269	total: 82.8ms	remaining: 5.83s
14:	learn: 0.4374252	total: 88.6ms	remaining: 5.82s
15:	learn: 0.4296899	total: 94.5ms	remaining: 5.81s
16:	learn: 0.4221040	total: 101ms	remaining: 5.83s
17:	learn: 0.4144123	total: 107ms	remaining: 5.83s
18:	learn: 0.4080156	total: 113ms	remaining: 5.83s

CatBoostError: /src/catboost/catboost/libs/data/model_dataset_compatibility.cpp:81: At position 8 should be feature with name year (found euribor3m).

#Test with answer set

In [ ]:
import os

root_dir = "/content/gdrive/MyDrive/Colab Notebooks/"

# Checking if our specified directory exists
os.path.exists(root_dir)

True

In [ ]:
# Paths to the downloaded files
data_path = os.path.join(root_dir, "merged_with_answer (1).csv")

# Load data
df_test = pd.read_csv(data_path)

In [ ]:
df_test.head()

,Unnamed: 0,age,job,marital,education,default,housing,loan,contact,month,...,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,id,key,y
0,0,54,admin.,married,university.degree,no,no,no,cellular,nov,...,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,24188,54admin.marrieduniversity.degreenononocellular...,no
1,1,36,entrepreneur,married,university.degree,no,yes,yes,telephone,jun,...,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1,8688,36entrepreneurmarrieduniversity.degreenoyesyes...,no
2,2,27,unemployed,married,high.school,no,yes,no,cellular,may,...,0,nonexistent,-1.8,92.893,-46.2,1.244,5099.1,35789,27unemployedmarriedhigh.schoolnoyesnocellularm...,no
3,3,39,admin.,married,university.degree,no,yes,no,cellular,aug,...,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,20034,39admin.marrieduniversity.degreenoyesnocellula...,no
4,4,43,services,married,high.school,unknown,yes,no,telephone,jun,...,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,11392,43servicesmarriedhigh.schoolunknownyesnoteleph...,no


In [ ]:
df_test

,Unnamed: 0,age,job,marital,education,default,housing,loan,contact,month,...,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,id,key,y
0,0,54,admin.,married,university.degree,no,no,no,cellular,nov,...,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,24188,54admin.marrieduniversity.degreenononocellular...,no
1,1,36,entrepreneur,married,university.degree,no,yes,yes,telephone,jun,...,0,nonexistent,1.4,94.465,-41.8,4.864,5228.1,8688,36entrepreneurmarrieduniversity.degreenoyesyes...,no
2,2,27,unemployed,married,high.school,no,yes,no,cellular,may,...,0,nonexistent,-1.8,92.893,-46.2,1.244,5099.1,35789,27unemployedmarriedhigh.schoolnoyesnocellularm...,no
3,3,39,admin.,married,university.degree,no,yes,no,cellular,aug,...,0,nonexistent,1.4,93.444,-36.1,4.965,5228.1,20034,39admin.marrieduniversity.degreenoyesnocellula...,no
4,4,43,services,married,high.school,unknown,yes,no,telephone,jun,...,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,11392,43servicesmarriedhigh.schoolunknownyesnoteleph...,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8343,9170,30,services,married,high.school,no,yes,no,telephone,nov,...,0,nonexistent,-0.1,93.200,-42.0,4.076,5195.8,26975,30servicesmarriedhigh.schoolnoyesnotelephoneno...,no
8344,9171,47,admin.,married,high.school,no,yes,no,telephone,may,...,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,6863,47admin.marriedhigh.schoolnoyesnotelephonemayw...,no
8345,9172,37,entrepreneur,married,basic.9y,no,no,no,telephone,jun,...,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,9449,37entrepreneurmarriedbasic.9ynononotelephoneju...,no
8346,9173,47,services,married,high.school,unknown,yes,no,telephone,may,...,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,916,47servicesmarriedhigh.schoolunknownyesnoteleph...,no


In [ ]:
df_test["month"] = df_test["month"].replace({"jul": 7, "jun": 6, "aug": 8, "may": 5, "oct" : 10, 'sep': 9, "nov": 11, "apr": 4, "mar":3, "dec": 12})

df_test["year"] = df_test["cons.price.idx"]
df_test["year"] = df_test["cons.price.idx"].replace({93.918 : 2008, 92.963 : 2009, 93.444 : 2008, 92.893 : 2009,
                                                       94.027 : 2010, 92.431 : 2009, 94.199 : 2010, 94.465 : 2008,
                                                       92.649 : 2009, 93.994 : 2008, 93.075 : 2009, 93.2 : 2008,
                                                       94.601 : 2010, 92.469 : 2009, 92.201 : 2009, 93.369 : 2010,
                                                       93.876 : 2010, 92.713 : 2009, 93.798 : 2008, 92.843 : 2009,
                                                       92.379 : 2009, 94.055 : 2010, 94.215 : 2010, 93.749 : 2010,
                                                       94.767 : 2010, 92.756 : 2008})

In [ ]:
df_test['job'].replace('unknown', df_test['job'].mode().iloc[0], inplace=True)

In [ ]:
df_test['loan'].replace('unknown', df_test['loan'].mode().iloc[0], inplace=True)

In [ ]:
df_test['marital'].replace('unknown', df_test['marital'].mode().iloc[0], inplace=True)

In [ ]:
df_test = df_test.drop(["key", "Unnamed: 0"], axis = 1)

In [ ]:
columns = df_test.select_dtypes(include=[object]).columns
df1 = pd.concat([df_test, pd.get_dummies(df_test[columns])], axis=1)
df1 = df1.drop(['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week','poutcome', 'y'], axis =1)
print("\n","Describe Dataset_v2","\n")
print(df1.info(),"\n \n \n")


 Describe Dataset_v2 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8348 entries, 0 to 8347
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            8348 non-null   int64  
 1   campaign                       8348 non-null   int64  
 2   pdays                          8348 non-null   int64  
 3   previous                       8348 non-null   int64  
 4   emp.var.rate                   8348 non-null   float64
 5   cons.price.idx                 8348 non-null   float64
 6   cons.conf.idx                  8348 non-null   float64
 7   euribor3m                      8348 non-null   float64
 8   nr.employed                    8348 non-null   float64
 9   id                             8348 non-null   int64  
 10  year                           8348 non-null   float64
 11  job_admin.                     8348 non-null   bool   
 12  job_blue-collar         

In [ ]:
df1.head()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,id,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,54,1,999,0,-0.1,93.200,-42.0,4.191,5195.8,24188,...,False,True,False,False,False,False,True,False,True,False
1,36,1,999,0,1.4,94.465,-41.8,4.864,5228.1,8688,...,False,False,False,False,True,False,True,False,True,False
2,27,4,999,0,-1.8,92.893,-46.2,1.244,5099.1,35789,...,False,True,False,False,False,False,True,False,True,False
3,39,4,999,0,1.4,93.444,-36.1,4.965,5228.1,20034,...,False,True,False,False,False,False,True,False,True,False
4,43,2,999,0,1.4,94.465,-41.8,4.959,5228.1,11392,...,True,False,False,False,False,False,True,False,True,False


In [ ]:
min_max_scaler=preprocessing.MinMaxScaler()
data_scaled1=pd.DataFrame(min_max_scaler.fit_transform(df1),columns=df1.columns)

In [ ]:
yt = data_scaled1.y_yes
xt = data_scaled1.drop(['y_yes','y_no'], axis = 1)

In [ ]:
xt.head()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,id,...,contact_cellular,contact_telephone,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success
0,0.480519,0.000000,1.0,0.0,0.687500,0.389322,0.368201,0.806349,0.877883,0.587318,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.246753,0.000000,1.0,0.0,1.000000,0.882307,0.376569,0.958957,1.000000,0.210894,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.129870,0.071429,1.0,0.0,0.333333,0.269680,0.192469,0.138095,0.512287,0.869053,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.285714,0.071429,1.0,0.0,1.000000,0.484412,0.615063,0.981859,1.000000,0.486437,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.337662,0.023810,1.0,0.0,1.000000,0.882307,0.376569,0.980499,1.000000,0.276562,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
xt.columns

Index(['age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'id',
       'year', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_basic.4y', 'education_basic.6y', 'education_basic.9y',
       'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'housing_no',
       'housing_unknown', 'housing_yes', 'loan_no', 'loan_yes',
       'contact_cellular', 'contact_telephone', 'day_of_week_fri',
       'day_of_week_mon', 'day_of_week_thu', 'day_of_week_tue',
       'day_of_week_wed', 'poutcome_failure', 'poutcome_nonexistent',
       'poutcome_success'],
      dty

In [ ]:
df.columns

Index(['id', 'age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'year', 'job_admin.',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'marital_divorced',
       'marital_married', 'marital_single', 'education_basic.4y',
       'education_basic.6y', 'education_basic.9y', 'education_high.school',
       'education_illiterate', 'education_professional.course',
       'education_university.degree', 'education_unknown', 'default_no',
       'default_unknown', 'housing_no', 'housing_unknown', 'housing_yes',
       'loan_no', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success'],
      dtype='object')

In [ ]:
xt = xt[['id', 'age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'year',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed',
       'marital_divorced', 'marital_married', 'marital_single', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_yes', 'contact_cellular', 'contact_telephone',
       'day_of_week_fri', 'day_of_week_mon', 'day_of_week_thu',
       'day_of_week_tue', 'day_of_week_wed', 'poutcome_failure',
       'poutcome_nonexistent', 'poutcome_success']]

In [ ]:
!pip install catboost
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

In [ ]:
# CatBoost 모델 정의
model = CatBoostClassifier()

#model training
model.fit(X_train, y_train)

y_final = model.predict(xt)

# F1 점수 계산
f1 = f1_score(yt, y_final)
print("F1 Score:", f1)

Learning rate set to 0.033205
0:	learn: 0.6641491	total: 5.94ms	remaining: 5.93s
1:	learn: 0.6377442	total: 11.5ms	remaining: 5.76s
2:	learn: 0.6136060	total: 17.2ms	remaining: 5.71s
3:	learn: 0.5911146	total: 22.9ms	remaining: 5.7s
4:	learn: 0.5708027	total: 28.6ms	remaining: 5.68s
5:	learn: 0.5514431	total: 34.3ms	remaining: 5.68s
6:	learn: 0.5341867	total: 39.9ms	remaining: 5.66s
7:	learn: 0.5184364	total: 45.7ms	remaining: 5.67s
8:	learn: 0.5036770	total: 51.3ms	remaining: 5.65s
9:	learn: 0.4904850	total: 56.2ms	remaining: 5.56s
10:	learn: 0.4778117	total: 63.5ms	remaining: 5.71s
11:	learn: 0.4668547	total: 69ms	remaining: 5.68s
12:	learn: 0.4566819	total: 74.5ms	remaining: 5.66s
13:	learn: 0.4467269	total: 80.1ms	remaining: 5.64s
14:	learn: 0.4374252	total: 85.9ms	remaining: 5.64s
15:	learn: 0.4296899	total: 91.6ms	remaining: 5.63s
16:	learn: 0.4221040	total: 97.3ms	remaining: 5.63s
17:	learn: 0.4144123	total: 103ms	remaining: 5.64s
18:	learn: 0.4080156	total: 109ms	remaining: 5.6


#Evaluation

In [ ]:
y_final = pd.Series(y_final)



In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

y_prob = model.predict_proba(xt)
print("Estimated probs:", y_prob[:10])

y_cls = model.predict(xt)
print("Estimated classes:", y_cls[:10])
print()

Estimated probs: [[0.89640459 0.10359541]
 [0.90825339 0.09174661]
 [0.63460615 0.36539385]
 [0.88164547 0.11835453]
 [0.94130309 0.05869691]
 [0.81055975 0.18944025]
 [0.93173032 0.06826968]
 [0.97057761 0.02942239]
 [0.94185811 0.05814189]
 [0.97775698 0.02224302]]
Estimated classes: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]



In [ ]:
print("Accuracy:", accuracy_score(yt, y_cls))
print("F1:", f1_score(yt, y_cls))
print("ROC AUC:", roc_auc_score(yt, y_prob[:, 1]))

Accuracy: 0.8633205558217537
F1: 0.47923322683706066
ROC AUC: 0.7827666190864601


In [ ]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(yt, y_cls)
conf_matrix_df = pd.DataFrame(
    conf_matrix,
    columns=["Predicted yes", "Predicted no"],
    index=["Actual yes", "Actual no"]
)
print(conf_matrix_df)

            Predicted yes  Predicted no
Actual yes           6682           674
Actual no             467           525


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(yt, y_cls))

              precision    recall  f1-score   support

         0.0       0.93      0.91      0.92      7356
         1.0       0.44      0.53      0.48       992

    accuracy                           0.86      8348
   macro avg       0.69      0.72      0.70      8348
weighted avg       0.88      0.86      0.87      8348

